# DATA PROCESSING

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation

# Read the data from the CSV file
data = pd.read_csv("./data/signals.csv")
# Identify the columns to format
columns_to_format = ["ACC_1", "ACC_2", "ACC_3"]  # Replace with your column names

# Define a function to format and clean the values
def format_value(value):
    if isinstance(value, str) and value.endswith('*""'):
        return float(value[:-3])
    return value

# Apply the format_value function to the specified columns
data[columns_to_format] = data[columns_to_format].applymap(format_value)

# Extract the relevant columns
acc_x = data["ACC_1"].astype(float).values
acc_y = data["ACC_2"].astype(float).values
acc_z = data["ACC_3"].astype(float).values
speed = data["SPEED"].astype(float).values
latitude = data["LATITUDE"].astype(float).values
longitude = data["LONGITUDE"].astype(float).values
timestamp = data["TIME"].astype(float).values

# Constants
g = 9.81  # Gravitational acceleration (m/s^2)
initial_period = 3  # Initial calibration period (seconds)

# Calculate the number of samples in the initial period
num_samples = len(data)
initial_samples = int(initial_period * num_samples / ((timestamp[-1] - timestamp[0]) * 1e-3))

# Calculate the average acceleration during the initial period
avg_acc_x = np.mean(acc_x[:initial_samples])
avg_acc_y = np.mean(acc_y[:initial_samples])
avg_acc_z = np.mean(acc_z[:initial_samples])

# Determine the initial orientation
# if abs(avg_acc_z) > abs(avg_acc_x) and abs(avg_acc_z) > abs(avg_acc_y):
initial_rotation = Rotation.from_euler('z', np.arctan2(avg_acc_y, avg_acc_x))
# else:
    # raise ValueError("Initial orientation not well-defined. Make sure the phone is stationary or moving at constant speed.")

# Initialize the transformed accelerations
transformed_acc = np.zeros((3, num_samples))

# Initialize the gyroscope orientation
gyro_rotation = Rotation.from_quat([1, 0, 0, 0])

# Process the data
for i in range(1, num_samples):
    # Calculate the change in heading using GPS coordinates
    dx = longitude[i] - longitude[i-1]
    dy = latitude[i] - latitude[i-1]
    heading_change = np.arctan2(dx, dy)
    
    # Update the gyroscope orientation (assuming gyroscope data is available)
    # gyro_rotation = gyro_rotation * Rotation.from_euler('z', heading_change)
    
    # Create the rotation matrix to align with the car's frame
    rotation_matrix = initial_rotation * gyro_rotation * Rotation.from_euler('z', -heading_change)
    
    # Transform the accelerations
    transformed_acc[:, i] = rotation_matrix.apply([acc_x[i], acc_y[i], acc_z[i]])

# Assign the transformed accelerations to the DataFrame
data["TRANSFORMED_ACC_X"] = transformed_acc[0, :]
data["TRANSFORMED_ACC_Y"] = transformed_acc[1, :]
data["TRANSFORMED_ACC_Z"] = transformed_acc[2, :]


# VISUALIZATION

In [2]:
import plotly.graph_objects as go

transformed_acc_x = data["TRANSFORMED_ACC_X"]
transformed_acc_y = data["TRANSFORMED_ACC_Y"]
transformed_acc_z = data["TRANSFORMED_ACC_Z"]

# Create the 3D scatter plot
fig = go.Figure()

# Add the original xyz accelerometer signals
fig.add_trace(go.Scatter3d(
    x=acc_x,
    y=acc_y,
    z=acc_z,
    mode='markers',
    marker=dict(size=2, color='blue'),
    name='Original'
))

# Add the transformed xyz accelerometer signals
fig.add_trace(go.Scatter3d(
    x=transformed_acc_x,
    y=transformed_acc_y,
    z=transformed_acc_z,
    mode='markers',
    marker=dict(size=2, color='red'),
    name='Transformed'
))

# Set the plot title and axis labels
fig.update_layout(
    title='Original vs Transformed Accelerometer Signals',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# Display the plot
fig.show()